Library import

In [7]:
from functions import A_preprocessing as preprocessing

In [8]:
df = preprocessing.read_dataset("data/raw_dataset/full_data_flightdelay.csv")

   MONTH  DAY_OF_WEEK  DEP_DEL15 DEP_TIME_BLK  DISTANCE_GROUP  SEGMENT_NUMBER  \
0      1            7          0    0800-0859               2               1   
1      1            7          0    0700-0759               7               1   
2      1            7          0    0600-0659               7               1   
3      1            7          0    0600-0659               9               1   
4      1            7          0    0001-0559               7               1   

   CONCURRENT_FLIGHTS  NUMBER_OF_SEATS            CARRIER_NAME  \
0                  25              143  Southwest Airlines Co.   
1                  29              191    Delta Air Lines Inc.   
2                  27              199    Delta Air Lines Inc.   
3                  27              180    Delta Air Lines Inc.   
4                  10              182        Spirit Air Lines   

   AIRPORT_FLIGHTS_MONTH  ...  PLANE_AGE       DEPARTING_AIRPORT  LATITUDE  \
0                  13056  ...         

In [9]:
df = preprocessing.split_departure_time_block_column(df)
df.head()

,MONTH,DAY_OF_WEEK,DEP_DEL15,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,...,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,DEP_START_TIME,DEP_END_TIME
0,1,7,0,2,1,25,143,Southwest Airlines Co.,13056,107363,...,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91,0800,0859
1,1,7,0,7,1,29,191,Delta Air Lines Inc.,13056,73508,...,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91,0700,0759
2,1,7,0,7,1,27,199,Delta Air Lines Inc.,13056,73508,...,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91,0600,0659
3,1,7,0,9,1,27,180,Delta Air Lines Inc.,13056,73508,...,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91,0600,0659
4,1,7,0,7,1,10,182,Spirit Air Lines,13056,15023,...,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91,0001,0559


In [11]:
df = preprocessing.drop_null_values(df)

AttributeError: module 'functions.A_preprocessing' has no attribute 'drop_null_values'